<a href="https://colab.research.google.com/github/seunghyeon98/News_RAG/blob/main/News_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchainhub
!pip install langchain
!pip install langchain_community
!pip install langchain_openai
!pip install --upgrade openai
!pip install faiss-cpu
!pip install langsmith
!pip install -U langchain langchainhub langchain-openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [22]:
import os

os.environ["OPENAI_API_KEY"] = "key"
# LangChain Hub API 키 설정
os.environ["LANGCHAIN_HUB_API_KEY"] = "key"
os.environ['LANGCHAIN_API_KEY'] = 'key'
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
# LangChain Hub의 프로젝트 설정
os.environ['LANGCHAIN_PROJECT'] = 'News-Rag'

In [9]:
# lang chain hub에 prompt push
from langchain import hub
from langchain.prompts.chat import ChatPromptTemplate

# prompt = ChatPromptTemplate.from_template(input_template)
# havi_rag repo에 prompt push
# hub.push("havi_rag", prompt, new_repo_is_public=False)

In [10]:

import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain.retrievers.multi_query import MultiQueryRetriever

# Text 로드 및 청킹

In [11]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/005/0001699583?ntype=RANKING",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body",
                             "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(page_content="\n18평 서울 아파트 ‘20년 전세’… 신혼부부 ‘파격 정책’\n\n\n올림픽파크포레온 300가구 시작공공임대주택 4396호 공급 추진무자녀 신혼부부도 입주 가능\n\n\n\n서울시가 임대주택으로 공급하는 강동구 둔촌동 '올림픽파크포레온' 전용 59㎡ A타입 주택 평면도. 올림픽파크포레온 제공서울시가 신혼부부 주거 부담을 해소하기 위해 장기전세주택 등 임대주택을 4000가구 이상 공급한다. 약 60㎡(약 18.15평·전용면적 기준) 크기 공공주택을 최장 20년간 전세로 사용할 수 있다. 자녀를 2명 이상 낳으면 시세에 비해 저렴하게 집을 매수할 권리인 ‘우선매수청구권’도 부여한다.신혼부부 위한 ‘20년 전세’… 무자녀도 가능 서울시는 29일 ‘저출생 대응 신혼부부 주택 확대방안’을 발표했다. 신혼부부를 위한 ‘장기전세주택 2’와 신혼부부 맞춤형 주거공간인 ‘신혼부부 안심주택’을 공급하는 내용으로, 올해부터 3년간 신혼부부에게 총 4396호의 공공주택을 공급하는 게 골자다.장기전세주택은 오세훈 시장이 재임 중이던 2007년 도입된 정책이다. 주변 전세 시세의 80% 이하 가격으로 최장 20년간 거주 가능한 임대주택이다. 기존에는 무자녀 신혼부부도 입주를 신청할 수 있었지만, 다자녀 가구에 우선권이 있어 출산하지 않은 이들은 입주가 사실상 불가능했다.장기전세주택 2는 기존 정책에서 무자녀 신혼부부에게 입주 기회를 확대하는 방향으로 재설계됐다. 결혼만 해도 기본 10년간 장기전세를 살 수 있고, 아이를 낳을 때마다 인센티브가 부여된다. 1자녀를 출산하면 재계약 시 소득 기준을 20% 완화해주고, 2자녀 이상 출산하면 전세로 살고 있는 집을 구매할 수 있는 ‘우선매수청구권’을 부여하는 식이다. 한 명이라도 자녀를 낳으면 계약기간이 최장 20년으로 늘어난다.\n\n\n\n장기전세주택 2 정책 개요. 서울시 제공‘올림픽파크포레온’ 300가구 우선공급입주 자격은 무주택 세대원으로 구성된 신혼부부다. 혼인신고일로부터 7년 이내 또

In [12]:
# chunk size, chunk overlap 값 설정에 대하여

# 정보의 연속성: 뉴스 기사는 문맥의 연속성이 중요합니다. 너무 작은 청크로 나누면 문맥이 손실될 수 있고, 너무 큰 청크는 모델의 입력 한계를 초과할 수 있다.

# chunk_size: 모델의 최대 입력 길이를 고려하여 적당한 크기로 설정,  예를 들어, 2048 토큰을 기준으로 하면 청크 크기를 1000-1500 글자 정도로 설정하는 것이 좋다.
# chunk_overlap: 청크 간에 문맥이 유지되도록 중첩 부분을 설정, 일반적으로 10-20% 정도의 중첩을 설정.
# 예를 들어, chunk_size를 1000으로 설정한 경우, chunk_overlap을 100-200 정도로 설정할 수 있다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

2

# Vector DB 설정

In [13]:



from langchain_community.embeddings import OpenAIEmbeddings
openai = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Faiss를 통해 벡터DB 설정
vectorstore = FAISS.from_documents(documents=splits, embedding=openai)

# Faiss를 통해 리트리버 생성
retriever = vectorstore.as_retriever()




/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


# Prompt Load

RAG를 통해 검색 및 응답을 받을때, 다음과 같은 Prompt를 설정하여 정확하고 간결한 정보를 제공받을 수 있게 설정한다.



In [14]:
# 나의 langchain hub의 Prompt 불러오기
prompt = hub.pull("havi_rag")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'havi_rag', 'lc_hub_commit_hash': '26f335ae9d0e8a97462ac6bb7c244a54935bf5cbcbf3ffd1746dc6f1915753f4'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [15]:
# 랭체인 허브에 있는 프롬프트 확인
print(prompt.messages[0].prompt.template)
input_template = prompt.messages[0].prompt.template

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [ ]:
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]

# RAG Chain 생성

In [16]:
from langchain.callbacks.base import BaseCallbackHandler

'''
BaseCallbackHandler
- 스트리밍 모드에서 모델이 생성하는 토큰을 실시간으로 처리하는 역할
- 모델이 새로운 토큰을 생성할 때마다,토큰을 실시간으로 처리하거나 출력할 수 있게 한다.
'''
class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)


llm = ChatOpenAI(
    # openai llm 모델 설정
    model_name="gpt-4-turbo-preview",
    # temperature : 얼마나 답변을 자유롭게 하느냐,
    # 0에 가까울 수록 정확하고 일관된 응답 : 코드 작성, 기술 문서 생성 task에 쓰임
    # 1에 가까울 수록 창의적이고 다양한 응답 : 스토리 생성, 브레인스토밍 등
    temperature=0,
    streaming=True,
    callbacks=[StreamCallback()], # StreamCallBack을 사용하여 실시간으로 모델의 출력을 볼 수 있게한다.
    openai_api_key = os.environ["OPENAI_API_KEY"]
)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 체인을 생성합니다.
'''
retriever
- 위에서 설정한 벡터 DB

RunnablePassthrough()
- invoke를 통해 사용자의 질의문을 전달

prompt
- 설정한 prompt

llm
- 설정한 llm

StrOutputParser()
 - LLM의 출력물(응답 결과)을 파싱할 형식 지정
 - 출력이 정리되고, 불필요한 공백이나 정보가 제거되어 간결한 형태로 제공된다.

'''
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



# 검색

In [17]:
# rag_chain  객체의  invoke 메서드를 사용하여 사용자의 질문을 처리한다.
single_query_answer = rag_chain.invoke(
    "신혼부부 안심주택이 뭐야?"
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

신혼부부 안심주택은 서울시가 신혼부부의 주거 부담을 해소하기 위해 공급하는 임대주택 모델로, 결혼 7년 이내의 신혼부부와 결혼 예정인 예비부부를 대상으로 합니다. 이 주택은 70%가 임대주택이고 30%가 분양주택으로 구성되며, 공공임대주택은 주변 시세의 50% 수준으로 공급됩니다. 신혼부부 맞춤형 공간과 서비스를 제공하며, 고급 빌트인 가전이 설치되고 역세권이나 간선도로변에 위치합니다.

In [ ]:
print(rag_chain.invoke(
    "부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?"
) )

부영그룹은 출산 직원에게 1억원을 지원합니다. 2021년 이후 태어난 직원 자녀에게 이 지원금을 제공하며, 연년생이나 쌍둥이 자녀가 있을 경우 총 2억원을 받게 됩니다. 셋째 아이를 낳는 경우에는 국민주택을 제공하겠다는 계획도 밝혔습니다.부영그룹은 출산 직원에게 1억원을 지원합니다. 2021년 이후 태어난 직원 자녀에게 이 지원금을 제공하며, 연년생이나 쌍둥이 자녀가 있을 경우 총 2억원을 받게 됩니다. 셋째 아이를 낳는 경우에는 국민주택을 제공하겠다는 계획도 밝혔습니다.


In [18]:
print(
    rag_chain.invoke("정부가 신혼부부를 대상으로 발표한 대책을 bullet points 형식으로 작성해 주세요.")
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

- 장기전세주택 2 정책: 서울시는 신혼부부 대상으로 '올림픽파크포레온' 등의 장기전세주택을 공급하며, 무주택 세대원으로 구성된 신혼부부(혼인신고일로부터 7년 이내 또는 6개월 이내 혼인신고 예정)가 대상이다.
- 신혼부부 안심주택: 결혼 7년 이내 신혼부부 및 결혼 예정 예비부부 대상으로, 70%는 임대, 30%는 분양주택으로 구성되며, 주변 시세의 50% 수준으로 공급된다. 신혼부부 맞춤형 공간과 고급 빌트인 가전이 제공되며, 위치는 역세권 또는 간선도로변에 건립된다.
- 장기전세주택 2와 신혼부부용 안심주택을 통해 연평균 4000가구 공급 예정, 무자녀 신혼부부도 입주 가능하며, 자녀 출산 시 재계약 소득 기준 완화 및 우선매수청구권 부여 등의 인센티브가 제공된다.- 장기전세주택 2 정책: 서울시는 신혼부부 대상으로 '올림픽파크포레온' 등의 장기전세주택을 공급하며, 무주택 세대원으로 구성된 신혼부부(혼인신고일로부터 7년 이내 또는 6개월 이내 혼인신고 예정)가 대상이다.
- 신혼부부 안심주택: 결혼 7년 이내 신혼부부 및 결혼 예정 예비부부 대상으로, 70%는 임대, 30%는 분양주택으로 구성되며, 주변 시세의 50% 수준으로 공급된다. 신혼부부 맞춤형 공간과 고급 빌트인 가전이 제공되며, 위치는 역세권 또는 간선도로변에 건립된다.
- 장기전세주택 2와 신혼부부용 안심주택을 통해 연평균 4000가구 공급 예정, 무자녀 신혼부부도 입주 가능하며, 자녀 출산 시 재계약 소득 기준 완화 및 우선매수청구권 부여 등의 인센티브가 제공된다.


# Multi Query Retriever 활용

In [ ]:
# 멀티 쿼리 리트리버 활용
mqr_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)


# 체인을 생성합니다.
rag_chain = (
    {"context": mqr_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# multiqueryretriever이 질문을 어떻게 재생성하는지 확인
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
question = "신혼부부 안심주택이 뭐야?"
multi_query_answer = rag_chain.invoke(question)



신혼부부를 위한 안심주택 제도에 대해 설명해 줄 수 있나요?

신혼부부 안심주택 프로그램의 자격 요건과 혜택은 무엇인가요?

신혼부부 안심주택에 관한 정보를 어디서 찾을 수 있나요?

INFO:langchain.retrievers.multi_query:Generated queries: ['신혼부부를 위한 안심주택 제도에 대해 설명해 줄 수 있나요?', '', '신혼부부 안심주택 프로그램의 자격 요건과 혜택은 무엇인가요?', '', '신혼부부 안심주택에 관한 정보를 어디서 찾을 수 있나요?']


신혼부부 안심주택은 서울시가 신혼부부의 주거 부담을 해소하기 위해 도입한 새로운 임대주택 모델입니다. 이 주택은 결혼 7년 이내의 신혼부부와 결혼 예정인 예비부부를 대상으로 하며, 70%는 임대로, 30%는 분양으로 구성됩니다. 공공임대주택은 주변 시세의 50% 수준으로 공급되며, 신혼부부 맞춤형 공간과 서비스가 제공됩니다.

## Single Query 와 Multi Query Retriever 검색 및 응답 결과 비교

In [ ]:
print('단일 쿼리 검색 결과:')
print(single_query_answer)
print()
print('멀티 쿼리 검색 결과:')
print(multi_query_answer)


단일 쿼리 검색 결과:
신혼부부 안심주택은 서울시가 신혼부부의 주거 부담을 줄이기 위해 도입한 새로운 임대주택 모델입니다. 이 모델은 결혼 7년 이내의 신혼부부와 결혼 예정인 예비부부를 대상으로 하며, 70%는 임대주택으로, 30%는 분양주택으로 구성됩니다. 공공임대주택은 주변 시세의 50% 수준으로 공급되며, 신혼부부 맞춤형 공간과 서비스가 제공됩니다.

멀티 쿼리 검색 결과:
신혼부부 안심주택은 서울시가 신혼부부의 주거 부담을 해소하기 위해 도입한 새로운 임대주택 모델입니다. 이 주택은 결혼 7년 이내의 신혼부부와 결혼 예정인 예비부부를 대상으로 하며, 70%는 임대로, 30%는 분양으로 구성됩니다. 공공임대주택은 주변 시세의 50% 수준으로 공급되며, 신혼부부 맞춤형 공간과 서비스가 제공됩니다.


In [ ]:
single_query_answer

'신혼부부 안심주택은 서울시가 신혼부부의 주거 부담을 줄이기 위해 도입한 새로운 임대주택 모델입니다. 이 모델은 결혼 7년 이내의 신혼부부와 결혼 예정인 예비부부를 대상으로 하며, 70%는 임대주택으로, 30%는 분양주택으로 구성됩니다. 공공임대주택은 주변 시세의 50% 수준으로 공급되며, 신혼부부 맞춤형 공간과 서비스가 제공됩니다.'

In [ ]:
multi_query_answer

'신혼부부 안심주택은 서울시가 신혼부부의 주거 부담을 해소하기 위해 도입한 새로운 임대주택 모델입니다. 이 주택은 결혼 7년 이내의 신혼부부와 결혼 예정인 예비부부를 대상으로 하며, 70%는 임대로, 30%는 분양으로 구성됩니다. 공공임대주택은 주변 시세의 50% 수준으로 공급되며, 신혼부부 맞춤형 공간과 서비스가 제공됩니다.'

In [ ]:
# Run
retriever = MultiQueryRetriever(
    retriever=vectordb.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.invoke(query="What does the course say about regression?")
len(unique_docs)

In [ ]:
# 컬렉션을 삭제합니다.
vectorstore.delete_collection()